## Study of the evolution of snowfall in the catalan Pyrenees

We will study the trend experienced in the snowfall pattern, from data of the daily evolution of the snow depth registered in various points in the catalan Pyrinees along the last decades. The main objective is then to study whether or not the total acumulated snowfall is decreasing.

The snow depth data is obtained from the catalan meteorological service (Servei Meteorològic de Catalunya) and the catalan geological and cartographic institute (Institut Geològic i Cartogràfic de Catalunya). The data registers the snow depth with a daily frequency, from November 1st to May 31st. In case of very premature snowfall, which has therefore not been registered, it will be assumed that the initial snowdepth is due to a single snowfall event.

In [1]:
# basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# libraries containing the developed classes and functions
import snowSeason as ss
import functions as f

ModuleNotFoundError: No module named 'snowSeason'

### Reanalysis

Due to the usually remoteness and harsh climatological conditions of the meteorological stations where the measurements are carried on, there might be days in which the data is incorrectly registered, or even no registered at all some days, such that the raw data contains invalid values of the snow depths. As we are interested in studying the snowfall episodes, this fact can yield an overestimation of it.

In order to interpolate the missing data, we consider a situation in which there are no additional snowfall episodes and hence that the snow depth steadily decreases. As a thermodynamic first order approximation, we assume the snow depth $D(t)$ to decrease at a rate inversely proportional to the current depth:
\begin{equation}
\frac{dD(t)}{dt} = -\frac{\kappa}{D(t)},
\end{equation}
with $\kappa$ a strictly positive constant defining the melting rate. This equation can be integrated to yield
\begin{equation}  
D(t) = \sqrt{D_0^2 - 2\kappa t},
\end{equation}
where $D_0$ is the starting depth. The melting rate is thus defined as $\kappa = (D_0^2 - D_f^2) / 2\Delta t$, where $\Delta t$ is the time spent for the snow depth to evolve from $D_0$ to $D_f < D_0$. The rate $\kappa$ will obviously depend on the instantaneous meteorological conditions, as well as on the microclimate characterising each measurement station, but we will assume, for the sake of simplicity, two different and constant values depending on whether the measurements are taken within the first part of the season (initial 70% of the days) or in the second part (last 30% of the days), as the temperatures tend to rapidly rise in the later. By inspection of various cases, we set a value of $\kappa = 150$ for the first part and $\kappa=400$ for the second one.

By applying this correction to the measured data we obtain much realistic evolutions:

In [ ]:
plt.figure(figsize = [10, 3])

plt.subplot(1, 2, 1)
plt.plot(snowDepth_LacRedon, label = 'raw data')
snowDepth_LacRedon = f.correctSnowDepth(snowDepth_LacRedon)    # correct data
plt.plot(snowDepth_LacRedon, '--', label = 'corrected data')
plt.ylabel('snow depth [cm]')
plt.xlabel('days since November 1st')
plt.legend(loc = 'upper right')
plt.title('Lac Redon 2247, 2009-2010')

plt.subplot(1, 2, 2)
plt.plot(snowDepth_PratdAguilo, label = 'raw data')
snowDepth_PratdAguilo = f.correctSnowDepth(snowDepth_PratdAguilo)    # correct data
plt.plot(snowDepth_PratdAguilo, '--', label = 'corrected data')
plt.ylabel('snow depth [cm]')
plt.xlabel('days since November 1st')
plt.legend()
plt.title('Cadí Nord - Prat d Aguiló 2143, 2008-2009')

plt.tight_layout()
plt.show()

Fig. 1) Time evolution of the snow depth in Lac Redon 2247 and Cadí Nord - Prat d'Aguiló 2143. Raw data (blue) and corrected data (orange). We observe how the rapid decrease and increase of the snow depth in both cases was clearly non-realistic, and would have yielded an overestimation of the snowfall.

### Read and correct data

In [ ]:
stations = ['Boí 2535', 'Bonaigua 2266', 'Cadí Nord - Prat d Aguiló 2143', 'Certascan 2400', 'El Port del Comte 2316',
            'Espot 2519', 'Lac Redon 2247', 'Malniu 2230', 'Núria 1971', 'Pastuira 2000', 'Salòria 2451', 'Sasseuva 2228',
            'Ulldeter 2364', 'Ulldeter 2410']


# run through all stations
stationDic = {}
for station in stations:
    
    # define path to read data from
    path = 'data/' + station + '.xlsx'
    
    # read data corrsponding to the current station
    data = pd.read_excel(path)
    keys = data.keys()

    # read dates
    dates = data[keys[0]]

    # run through all seasons
    seasonDic = {}
    for seasonYear in keys[1:]:
        seasonData = data[seasonYear]

        # read snow depths for each season
        snowDepth = np.zeros(len(seasonData)) 
        for i in range(len(seasonData)):
            if (type(seasonData[i]) != str):
                if not (np.isnan(seasonData[i])):
                    snowDepth[i] = seasonData[i]


        # correct snow depths
        snowDepth = f.correctSnowDepth(snowDepth)

        # store season in a dictionary, with years as key
        seasonDic[seasonYear] = ss.snowSeason(snowDepth, dates)
        
    
    # store seasonDic in a dictionary, with station as key
    stationDic[station] = seasonDic